# Deep Learning

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from models import Bin, Five
import torch
from torch.utils.data import Dataset, DataLoader
import onnxruntime as ort

/home/hoangphuc/miniconda3/envs/gene/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint_bin = "checkpoints/ckp_bin.pt"
checkpoint_last_bin = "checkpoints/ckp_bin_last.pt"
checkpoint_five = "checkpoints/ckp_five.pt"
checkpoint_last_five = "checkpoints/ckp_five_last.pt"
modelBin = Bin()
checkpoint_dict = torch.load(checkpoint_bin, map_location='cpu')
print("Load model control or diseases")
modelBin.load_state_dict(checkpoint_dict['net'])
modelBin.to('cuda:0')
print("f1 score: ", checkpoint_dict['f1'])

modelFive = Five()
checkpoint_dict = torch.load(checkpoint_five, map_location='cpu')
print("Load model five diseases")
modelFive.load_state_dict(checkpoint_dict['net'])
modelFive.to('cuda:0')
print("f1 score: ", checkpoint_dict['f1'])

input_names = ["gene"]
output_names = ["result"]
dummy = torch.randn(1,8630, device='cuda')
modelBin.eval()
torch.onnx.export(modelBin, dummy, "checkpoints/Bin.onnx", input_names=input_names, output_names=output_names, verbose=True)

modelFive.eval()
torch.onnx.export(modelFive, dummy, "checkpoints/Five.onnx", input_names=input_names, output_names=output_names, verbose=True)

Load model control or diseases
f1 score:  1.0
Load model five diseases
f1 score:  0.7308513708513709
Exported graph: graph(%gene : Float(1, 8630, strides=[8630, 1], requires_grad=0, device=cuda:0),
      %fc1.weight : Float(8192, 8630, strides=[8630, 1], requires_grad=1, device=cuda:0),
      %fc1.bias : Float(8192, strides=[1], requires_grad=1, device=cuda:0),
      %layer_out.weight : Float(2, 8192, strides=[8192, 1], requires_grad=1, device=cuda:0),
      %layer_out.bias : Float(2, strides=[1], requires_grad=1, device=cuda:0),
      %batchnorm1.weight : Float(8192, strides=[1], requires_grad=1, device=cuda:0),
      %batchnorm1.bias : Float(8192, strides=[1], requires_grad=1, device=cuda:0),
      %batchnorm1.running_mean : Float(8192, strides=[1], requires_grad=0, device=cuda:0),
      %batchnorm1.running_var : Float(8192, strides=[1], requires_grad=0, device=cuda:0)):
  %/fc1/Gemm_output_0 : Float(1, 8192, strides=[8192, 1], requires_grad=1, device=cuda:0) = onnx::Gemm[alpha=1., b

In [3]:
ckp_bin = "checkpoints/Bin.onnx"
ckp_five = "checkpoints/Five.onnx"

class2idxBin = {
    "control":0,
    "diseases":1,
}

idx2classBin = {v: k for k, v in class2idxBin.items()}

class2idxFive = {
    "CRC":0,
    "Lung":1,
    "Gastric":2,
    "Liver":3,
    "Breast":4
}

idx2classFive = {v: k for k, v in class2idxFive.items()}

class2idxSix = {
    "CRC":0,
    "Control":1,
    "Lung":2,
    "Gastric":3,
    "Liver":4,
    "Breast":5
}

idx2classSix = {v: k for k, v in class2idxSix.items()}

testloader = []

with open('data/val_bin.npy', 'rb') as f:
    x_test0 = np.load(f)
with open('data/val_bin_target.npy', 'rb') as f:
    y_test0 = np.load(f)

with open('data/test_bin.npy', 'rb') as f:
    x_test1 = np.load(f)
with open('data/test_bin_target.npy', 'rb') as f:
    y_test1 = np.load(f)

x_test_bin = np.concatenate([x_test0, x_test1])
y_test_bin = np.concatenate([y_test0, y_test1])

with open('data/val_five.npy', 'rb') as f:
    x_test0 = np.load(f)
with open('data/val_five_target.npy', 'rb') as f:
    y_test0 = np.load(f)

with open('data/test_five.npy', 'rb') as f:
    x_test1 = np.load(f)
with open('data/test_five_target.npy', 'rb') as f:
    y_test1 = np.load(f)

x_test_five = np.concatenate([x_test0, x_test1])
y_test_five = np.concatenate([y_test0, y_test1])

x_test = x_test_five.copy()
y_test = y_test_five.copy()

for i,v in enumerate(y_test_five):
    if v > 0:
        y_test[i] += 1

for i,v in enumerate(y_test_bin):
    if v == 0:
        np.vstack([x_test, x_test_bin[i]])
        y_test = np.append(y_test, 1)

randomize = np.arange(len(x_test))
np.random.shuffle(randomize)
x_test = x_test[randomize]
y_test = y_test[randomize]
x_test = np.array(x_test, np.float32)

class ClassifierDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


test_dataset = ClassifierDataset(torch.from_numpy(x_test).float(), torch.from_numpy(y_test).long())

testloader = DataLoader(dataset=test_dataset, batch_size=1)

cuda=True
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] if cuda else ['CPUExecutionProvider']
sessionbin = ort.InferenceSession(ckp_bin, providers=providers)
sessionfive = ort.InferenceSession(ckp_five, providers=providers)


/home/hoangphuc/miniconda3/envs/gene/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


In [4]:
inname = [i.name for i in sessionbin.get_inputs()]
inname

['gene']

In [5]:
outname = [i.name for i in sessionbin.get_outputs()]
outname

['result']

In [6]:
idx = 30
x = np.expand_dims(x_test[idx], axis=0)
print(x)
print(y_test[idx])
print("result:", idx2classSix[y_test[idx]])

[[0.43747813 0.4488727  0.6117503  ... 0.63823336 0.44232342 0.51017886]]
3
result: Gastric


In [7]:
x = np.expand_dims(x_test[idx], axis=0)

out = sessionbin.run(outname,{'gene':x})
result = np.argmax(out)
print("Control or disease: ",idx2classBin[result])
if result == 1:
    out = sessionfive.run(outname,{'gene':x})
    result = np.argmax(out)
    print("diseases: ",idx2classFive[result])


Control or disease:  diseases
diseases:  Gastric


# Machine Learning

In [8]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

In [9]:
# load the model from disk
filename = 'checkpoints/ml_svm_bin.sav'
svm_bin = pickle.load(open(filename, 'rb'))
filename = 'checkpoints/ml_rf_bin.sav'
rf_bin = pickle.load(open(filename, 'rb'))
filename = 'checkpoints/ml_knn_bin.sav'
knn_bin = pickle.load(open(filename, 'rb'))

filename = 'checkpoints/ml_svm_five.sav'
svm_five = pickle.load(open(filename, 'rb'))
filename = 'checkpoints/ml_rf_five.sav'
rf_five = pickle.load(open(filename, 'rb'))
filename = 'checkpoints/ml_knn_five.sav'
knn_five = pickle.load(open(filename, 'rb'))

class2idxBin = {
    "control":0,
    "diseases":1,
}

idx2classBin = {v: k for k, v in class2idxBin.items()}

class2idxFive = {
    "CRC":0,
    "Lung":1,
    "Gastric":2,
    "Liver":3,
    "Breast":4
}

idx2classFive = {v: k for k, v in class2idxFive.items()}

class2idxSix = {
    "CRC":0,
    "Control":1,
    "Lung":2,
    "Gastric":3,
    "Liver":4,
    "Breast":5
}

idx2classSix = {v: k for k, v in class2idxSix.items()}

testloader = []

with open('data/val_bin.npy', 'rb') as f:
    x_test0 = np.load(f)
with open('data/val_bin_target.npy', 'rb') as f:
    y_test0 = np.load(f)

with open('data/test_bin.npy', 'rb') as f:
    x_test1 = np.load(f)
with open('data/test_bin_target.npy', 'rb') as f:
    y_test1 = np.load(f)

x_test_bin = np.concatenate([x_test0, x_test1])
y_test_bin = np.concatenate([y_test0, y_test1])

with open('data/val_five.npy', 'rb') as f:
    x_test0 = np.load(f)
with open('data/val_five_target.npy', 'rb') as f:
    y_test0 = np.load(f)

with open('data/test_five.npy', 'rb') as f:
    x_test1 = np.load(f)
with open('data/test_five_target.npy', 'rb') as f:
    y_test1 = np.load(f)

x_test_five = np.concatenate([x_test0, x_test1])
y_test_five = np.concatenate([y_test0, y_test1])

x_test = x_test_five.copy()
y_test = y_test_five.copy()
for i,v in enumerate(y_test_five):
    if v > 0:
        y_test[i] += 1

for i,v in enumerate(y_test_bin):
    if v == 0:
        np.vstack([x_test, x_test_bin[i]])
        y_test = np.append(y_test, 1)

randomize = np.arange(len(x_test))
np.random.shuffle(randomize)
x_test = x_test[randomize]
y_test = y_test[randomize]

In [10]:
idx = 0
x = np.expand_dims(x_test[idx], axis=0)
print(x)
print(y_test[idx])
print("result:", idx2classSix[y_test[idx]])

[[0.34826578 0.30206029 0.57796861 ... 0.42200554 0.34917635 0.37751871]]
5
result: Breast


In [11]:
x = np.expand_dims(x_test[idx], axis=0)

result = svm_bin.predict(x)[0]
print("Control or disease: ",idx2classBin[result])
if result == 1:
    result = svm_five.predict(x)[0]
    print("diseases: ",idx2classFive[result])

Control or disease:  diseases
diseases:  Breast


In [12]:
x = np.expand_dims(x_test[idx], axis=0)

result = rf_bin.predict(x)[0]
print("Control or disease: ",idx2classBin[result])
if result == 1:
    result = rf_five.predict(x)[0]
    print("diseases: ",idx2classFive[result])

Control or disease:  diseases
diseases:  Breast


In [13]:
x = np.expand_dims(x_test[idx], axis=0)

result = knn_bin.predict(x)[0]
print("Control or disease: ",idx2classBin[result])
if result == 1:
    result = knn_five.predict(x)[0]
    print("diseases: ",idx2classFive[result])

Control or disease:  diseases
diseases:  Breast
